<a href="https://colab.research.google.com/github/Akasxh/Terrain-Recognition/blob/main/final_terrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#make sure to have the files of test and training
#here I have used google drive conviniently with my collab

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
with open('/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat '/gdrive/My Drive/foo.txt'

Hello Google Drive!

In [5]:
import numpy as np
from keras.preprocessing import image

In [ ]:
# 'Sequential' library used to Initialize NN as sequence of layers (Alternative to Graph initialization)
from keras.models import Sequential
# 'Conv2D' for 1st step of adding convolution layers to images ('Conv3D' for videos with time as 3rd dimension)
from keras.layers import Conv2D
# 'MaxPooling2D' step 2 for pooling of max values from Convolution Layers
from keras.layers import MaxPooling2D
# 'Flatten' Pooled Layers for step 3
from keras.layers import Flatten
# 'Dense' for fully connected layers that feed into classic ANN
from keras.layers import Dense
# Load saved Model
from keras.models import load_model

In [6]:
# Convolution Neural Network

# Initializing the CNN
classifier = Sequential()

# Step 1 - Convolution
# 'Conv2D' (Number of Filters, (Filter Row, Filter Column), input shape of inputs = (3 color channels, 64x64 -> 256x256 dimension of 2D array in each channel))
# Start with 32 filters, work your way up to 64 -> 128 -> 256
# 'input_shape' needs all picture inputs to be the same shape and format (2D array for B&W, 3D for Color images with each 2D array channel being Blue/Green/Red)
# 'input_shape' parameter shape matters (3,64,64) vs (64,64,3)
# 'Relu' Rectifier Activation Function used to get rid of -ve pixel values and increase non-linearity
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
# Preserves Spatial Structure and performance of model while reducing computation time
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolution layer to improve performance
# Only need 'input_shape' for Input Layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
# Take all the Pooled Feature Maps and put them into one huge single Vector that will input into a classic NN
classifier.add(Flatten())

# Step 4 - Full connection
# 'Activation' function makes sure relevant Nodes get a stronger vote or no vote
classifier.add(Dense(units = 128, activation = 'relu'))
# Add final Output Layer with five options
classifier.add(Dense(units = 5, activation = 'softmax'))

# Compiling the CNN
# 'adam' Stochastic Gradient Descent optimizer
# 'loss' function. Logarithmic loss 'categorical_crossentropy' for many objects here five

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [ ]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

# Create random transformation from Data to increase Dataset and prevent overfitting
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
training_set = train_datagen.flow_from_directory('/gdrive/MyDrive/Training Data',
                                                 target_size = (64, 64),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')


test_set = test_datagen.flow_from_directory('/gdrive/MyDrive/Testing Data',
                                            target_size = (64, 64),
                                            batch_size = 64,
                                            class_mode = 'categorical')

Found 10000 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [ ]:
##model.add(Dense(5, activation='softmax'))


In [10]:
# Train the model
# Increase 'epochs' to boost model performance (takes longer)
classifier.fit_generator(training_set,
                         steps_per_epoch = 156,
                         epochs = 10,
                         validation_data = test_set,
                         validation_steps = 7)

<ipython-input-10-5ac0c837cc52>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set,


Epoch 1/10
156/156 [==============================] - 2515s 16s/step - loss: 0.9158 - accuracy: 0.6367 - val_loss: 0.7944 - val_accuracy: 0.7076
Epoch 2/10
156/156 [==============================] - 31s 201ms/step - loss: 0.7791 - accuracy: 0.7019 - val_loss: 0.7881 - val_accuracy: 0.6853
Epoch 3/10
156/156 [==============================] - 31s 200ms/step - loss: 0.7477 - accuracy: 0.7170 - val_loss: 0.8297 - val_accuracy: 0.6875
Epoch 4/10
156/156 [==============================] - 33s 211ms/step - loss: 0.7098 - accuracy: 0.7365 - val_loss: 0.7262 - val_accuracy: 0.7232
Epoch 5/10
156/156 [==============================] - 32s 202ms/step - loss: 0.6951 - accuracy: 0.7379 - val_loss: 0.7663 - val_accuracy: 0.7277
Epoch 6/10
156/156 [==============================] - 31s 200ms/step - loss: 0.6569 - accuracy: 0.7519 - val_loss: 0.6711 - val_accuracy: 0.7522
Epoch 7/10
156/156 [==============================] - 31s 200ms/step - loss: 0.6373 - accuracy: 0.7614 - val_loss: 0.6795 - val_ac

In [11]:
# Save model to file
# Architecture of the model, allowing to reuse trained models
# Weights of the model
# Training configuration (loss, optimizer)
# State of the optimizer, allowing to resume training exactly where you left off
classifier.save('/gdrive/MyDrive/models/sih_model_y.h5')

# Examine model
classifier.summary()

# Examine Weights
classifier.weights

# Examine Optimizer
classifier.optimizer


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### *Now we have completed the model. We call the model now and test it with a image that it was not trained with.*


In [12]:
model = load_model('/gdrive/MyDrive/models/sih_model_y.h5')

In [13]:
test_image = image.load_img('/gdrive/MyDrive/Validation Data/Desert/Desert-Valid (1).jpeg', target_size = (64, 64))
# Add a 3rd Color dimension to match Model expectation
test_image = image.img_to_array(test_image)
# Add one more dimension to beginning of image array so 'Predict' function can receive it (corresponds to Batch, even if only one batch)
test_image = np.expand_dims(test_image, axis = 0)

In [14]:
result = classifier.predict(test_image)

1/1 [==============================] - 0s 153ms/step


In [18]:
# We now need to pull up the mapping
training_set.class_indices

print("The model class indices are:", training_set.class_indices)

The model class indices are: {'Coast': 0, 'Desert': 1, 'Forest': 2, 'Glacier': 3, 'Mountain': 4}


In [16]:
class_labels = ['Coast', 'Desert', 'Forest', 'Glacier', 'Mountain']

# Get the index of the predicted class
predicted_class_index = np.argmax(result, axis=1)[0]

# Use the index to get the class label
prediction = class_labels[predicted_class_index]

In [17]:
print("Prediction: " + prediction)


Prediction: class2
